In [2]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random

import simpy


RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
#---------------------------------------------------------------------------------------------------------------------------        
        #Tiempo de espera para que se genere el siguiente cliente
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


In [4]:
"""
Carwash example.

Covers:

- Waiting for other processes
- Resources: Resource

Scenario:
  A carwash has a limited number of washing machines and defines
  a washing processes that takes some (random) time.

  Car processes arrive at the carwash at a random time. If one washing
  machine is available, they start the washing process and wait for it
  to finish. If not, they wait until they an use one.

"""
import random

import simpy


RANDOM_SEED = 42
NUM_MACHINES = 2  # Número de máquinas en el lavado de autos
WASHTIME = 5      # Minutos que toma limpiar un auto
T_INTER = 7       # Crea un auto cada ~ 7 minutos
SIM_TIME = 20    # Tiempo de simulación en minutos


class Carwash(object):
    """Un lavado de autos tiene un número limitado de máquinas (`` NUM_MACHINES``) 
    para limpiar autos en paralelo.

Los autos tienen que solicitar una de las máquinas. Cuando tienen uno, pueden comenzar 
los procesos de lavado y esperar a que termine (lo que requiere ``washtime'').

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """Los procesos de lavado. Toma un proceso `` car`` e 
        intenta limpiarlo."""
        yield self.env.timeout(WASHTIME)
        print("Carwash elimino %d%% del %s de suciedad." %(random.randint(50, 99), car))


def car(env, name, cw):
    """El proceso del automóvil (cada automóvil tiene un `` nombre``) 
    llega al lavado de autos (`` cw``) y solicita una máquina de limpieza.

     A continuación, comienza el proceso de lavado, espera a que 
     termine y se va para nunca volver...

    """
    print('%s llega al lavado de autos en %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra al lavado de autos en %.2f.' % (name, env.now))
        yield env.process(cw.wash(name)) #ejecuta el proceso lavar auto

        print('%s deja el lavado de autos en %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Crea un lavado de autos, una cantidad de autos iniciales y 
    sigue creando autos aprox. cada `` t_inter`` minutos."""
    # Crea el lavado de autos
    carwash = Carwash(env, num_machines, washtime)

    # Crea 4 autos iniciales
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Crea más autos mientras se ejecuta la simulación
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))


# Setup and start the simulation
print('Carwash')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute!
env.run(until=SIM_TIME)

Carwash
Carro 0 llega al lavado de autos en 0.00.
Carro 1 llega al lavado de autos en 0.00.
Carro 2 llega al lavado de autos en 0.00.
Carro 3 llega al lavado de autos en 0.00.
Carro 0 entra al lavado de autos en 0.00.
Carro 1 entra al lavado de autos en 0.00.
Carro 4 llega al lavado de autos en 5.00.
Carwash elimino 97% del Carro 0 de suciedad.
Carwash elimino 67% del Carro 1 de suciedad.
Carro 0 deja el lavado de autos en 5.00.
Carro 1 deja el lavado de autos en 5.00.
Carro 2 entra al lavado de autos en 5.00.
Carro 3 entra al lavado de autos en 5.00.
Carro 5 llega al lavado de autos en 10.00.
Carwash elimino 64% del Carro 2 de suciedad.
Carwash elimino 58% del Carro 3 de suciedad.
Carro 2 deja el lavado de autos en 10.00.
Carro 3 deja el lavado de autos en 10.00.
Carro 4 entra al lavado de autos en 10.00.
Carro 5 entra al lavado de autos en 10.00.
Carwash elimino 97% del Carro 4 de suciedad.
Carwash elimino 56% del Carro 5 de suciedad.
Carro 4 deja el lavado de autos en 15.00.
Carro 5

In [ ]:
"""
Machine shop example

Covers:

- Interrupts
- Resources: PreemptiveResource

Scenario:
  A workshop has *n* identical machines. A stream of jobs (enough to
  keep the machines busy) arrives. Each machine breaks down
  periodically. Repairs are carried out by one repairman. The repairman
  has other, less important tasks to perform, too. Broken machines
  preempt theses tasks. The repairman continues them when he is done
  with the machine repair. The workshop works continuously.

"""
import random

import simpy


RANDOM_SEED = 42
PT_MEAN = 10.0         # Avg. processing time in minutes
PT_SIGMA = 2.0         # Sigma of processing time
MTTF = 300.0           # Mean time to failure in minutes
BREAK_MEAN = 1 / MTTF  # Param. for expovariate distribution
REPAIR_TIME = 30.0     # Time it takes to repair a machine in minutes
JOB_DURATION = 30.0    # Duration of other jobs in minutes
NUM_MACHINES = 10      # Number of machines in the machine shop
WEEKS = 4              # Simulation time in weeks
SIM_TIME = WEEKS * 7 * 24 * 60  # Simulation time in minutes


def time_per_part():
    """Devuelve el tiempo de procesamiento real para una pieza en concreto."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Retorna el tiempo hasta la siguiente falla para una máquina."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """Una máquina produce partes y se rompen de vez en cuando..

    Si se rompe, solicita un * repairman * y continúa la producción 
    después de que se repare.

    Una máquina tiene un *name* y un número de *parts_made* hasta el momento.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Comienzan los procesos "working" y "break_machine" para esta máquina.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce piezas siempre que se ejecute la simulación..

        Mientras hace una parte, la máquina puede romperse varias veces.
        Solicite un reparador cuando esto ocurra.

        """
        while True:
            #Comienza a hacer una nueva parte
            done_in = time_per_part()
            while done_in:
                try:
                    # Trabajando en la parte
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Establezca 0 para salir del bucle while

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # ¿Cuanto tiempo falta?

                    #Solicite un reparador. Esto adelantará su "other_job".
                    with repairman.request(priority=1) as req:#deja su trabajo menos prioritario
                        yield req
                        yield self.env.timeout(REPAIR_TIME)#repara la máquina

                    self.broken = False#maquina reparada

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """Rompe la máquina de vez en cuando."""
        while True:
            yield self.env.timeout(time_to_failure())#produce un fallo cada x tiempo
            if not self.broken: #si no está fallando
                # Solo rompe la máquina si está funcionando actualmente..
                self.process.interrupt()


def other_jobs(env, repairman):
    """El otro trabajo del reparador (sin importancia)."""
    while True:
        # Comienza un nuevo trabajo
        done_in = JOB_DURATION
        while done_in:
            # Vuelva a intentar el trabajo hasta que esté listo.
            # Su prioridad es menor que la reparación de la máquina.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start


# Setup and start the simulation
print('Machine shop')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Máquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Execute!
env.run(until=SIM_TIME)

# Analyis/results
print('Resultados de la tienda de máquinas después de %s semanas' % WEEKS)
for machine in machines:
    print('%s hizo %d partes.' % (machine.name, machine.parts_made))

In [1]:
"""
Event Latency example

Covers:

- Resources: Store

Scenario:
  This example shows how to separate the time delay of events between
  processes from the processes themselves.

When Useful:
  When modeling physical things such as cables, RF propagation, etc.  it
  better encapsulation to keep this propagation mechanism outside of the
  sending and receiving processes.

  Can also be used to interconnect processes sending messages

Example by:
  Keith Smith

"""
import simpy


SIM_DURATION = 100


class Cable(object):
    """This class represents the propagation through a cable."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(5)
        cable.put('Sender sent this at %d' % env.now)


def receiver(env, cable):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield cable.get()
        print('Received this at %d while %s' % (env.now, msg))


# Setup and start the simulation
print('Event Latency')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

Event Latency
Received this at 15 while Sender sent this at 5
Received this at 20 while Sender sent this at 10
Received this at 25 while Sender sent this at 15
Received this at 30 while Sender sent this at 20
Received this at 35 while Sender sent this at 25
Received this at 40 while Sender sent this at 30
Received this at 45 while Sender sent this at 35
Received this at 50 while Sender sent this at 40
Received this at 55 while Sender sent this at 45
Received this at 60 while Sender sent this at 50
Received this at 65 while Sender sent this at 55
Received this at 70 while Sender sent this at 60
Received this at 75 while Sender sent this at 65
Received this at 80 while Sender sent this at 70
Received this at 85 while Sender sent this at 75
Received this at 90 while Sender sent this at 80
Received this at 95 while Sender sent this at 85


In [7]:
"""
Movie renege example

Covers:

- Resources: Resource
- Condition events
- Shared events

Scenario:
  A movie theatre has one ticket counter selling tickets for three
  movies (next show only). When a movie is sold out, all people waiting
  to buy tickets for that movie renege (leave queue).

"""
import collections
import random

import simpy


RANDOM_SEED = 42
TICKETS = 50  # Número de boletos por pelicula
SIM_TIME = 120  # Simular hasta

def  moviegoer ( env , movie ,  num_tickets ,  theater): 
    """Un espectador intenta por una cantidad de boletos (* num_tickets *) para 
    una cierta * película * en un * teatro *. 

    Si la película se agota, ella sale del teatro. Si ella llega 
    al mostrador, ella trata de comprar un número de boletos. Si no 
    quedan suficientes boletos, ella discute con el cajero y se va. 
    Si a lo sumo se deja un boleto después de que el espectador haya comprado sus 
    boletos, el evento * sold out * esta película se desencadena causando que 
    todos los espectadores restantes se vayan
    """ 
    with theater.counter.request() as my_turn:
        # Espera hasta que sea nuestro turno o hasta que la película se haya agotado
        result = yield my_turn | theater.sold_out[movie]

        # Comprueba si es nuestro turno o si la película está agotada
        if my_turn not in result:
            theater.num_renegers[movie] += 1 #la persona abandona la fila si están agotadas
            env.exit()

        # Verifica si quedan suficientes boletos.
        if theater.available[movie] < num_tickets:
            # El espectador se va después de una discusión
            yield env.timeout(0.5)
            env.exit()

        # Comprar boletos
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2: #se activa la señal de agotado
            # Dispara el evento "agotado" para la película
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)
        
def customer_arrivals(env, theater):
    """Crea nuevos * moviegoers * hasta que el tiempo de simulación llegue a 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')


# Setup and start the simulation
print('Movie renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Create movie theater
counter = simpy.Resource(env, capacity=1)
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

# Start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analysis/results
for movie in movies:
    if theater.sold_out[movie]:
        print('La pelicula "%s" se vendio %.1f minutos después de la apertura del mostrador de boletos. '
              % (movie, theater.when_sold_out[movie]))
        print('Número de personas que abandonan la cola cuando la película se agotó: %s' %
              theater.num_renegers[movie])

Movie renege
La pelicula "Python Unchained" se vendio 38.0 minutos después de la apertura del mostrador de boletos. 
Número de personas que abandonan la cola cuando la película se agotó: 16
La pelicula "Kill Process" se vendio 43.0 minutos después de la apertura del mostrador de boletos. 
Número de personas que abandonan la cola cuando la película se agotó: 5
La pelicula "Pulp Implementation" se vendio 28.0 minutos después de la apertura del mostrador de boletos. 
Número de personas que abandonan la cola cuando la película se agotó: 5


In [3]:
"""
Process communication example

Covers:

- Resources: Store

Scenario:
  This example shows how to interconnect simulation model elements
  together using :class:`~simpy.resources.store.Store` for one-to-one,
  and many-to-one asynchronous processes. For one-to-many a simple
  BroadCastPipe class is constructed from Store.

When Useful:
  When a consumer process does not always wait on a generating process
  and these processes run asynchronously. This example shows how to
  create a buffer and also tell is the consumer process was late
  yielding to the event from a generating process.

  This is also useful when some information needs to be broadcast to
  many receiving processes

  Finally, using pipes can simplify how processes are interconnected to
  each other in a simulation model.

Example By:
  Keith Smith

"""
import random

import simpy


RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """A Broadcast pipe that allows one process to send messages to many.

    This construct is useful when message consumers are running at
    different rates than message generators and provides an event
    buffering to the consuming processes.

    The parameters are used to create a new
    :class:`~simpy.resources.store.Store` instance each time
    :meth:`get_output_conn()` is called.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Broadcast a *value* to all receivers."""
        if not self.pipes:
            raise RuntimeError('There are no output pipes.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        """Get a new output connection for this broadcast pipe.

        The return value is a :class:`~simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        # messages are time stamped to later check if the consumer was
        # late getting them.  Note, using event.triggered to do this may
        # result in failure due to FIFO nature of simulation yields.
        # (i.e. if at the same env.now, message_generator puts a message
        # in the pipe first and then message_consumer gets from pipe,
        # the event.triggered will be True in the other order it will be
        # False
        msg = (env.now, '%s says hello at %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # if message was already put into pipe, then
            # message_consumer was late getting to it. Depending on what
            # is being modeled this, may, or may not have some
            # significance
            print('LATE Getting Message: at time %d: %s received message: %s' %
                  (env.now, name, msg[1]))

        else:
            # message_consumer is synchronized with message_generator
            print('at time %d: %s received message: %s.' %
                  (env.now, name, msg[1]))

        # Process does some other work, which may result in missing messages
        yield env.timeout(random.randint(4, 8))


# Setup and start the simulation
print('Process communication')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# For one-to-one or many-to-one type pipes, use Store
pipe = simpy.Store(env)
env.process(message_generator('Generator A', env, pipe))
env.process(message_consumer('Consumer A', env, pipe))

print('\nOne-to-one pipe communication\n')
env.run(until=SIM_TIME)

# For one-to many use BroadcastPipe
# (Note: could also be used for one-to-one,many-to-one or many-to-many)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generator A', env, bc_pipe))
env.process(message_consumer('Consumer A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumer B', env, bc_pipe.get_output_conn()))

print('\nOne-to-many pipe communication\n')
env.run(until=SIM_TIME)

Process communication

One-to-one pipe communication

at time 6: Consumer A received message: Generator A says hello at 6.
at time 12: Consumer A received message: Generator A says hello at 12.
at time 19: Consumer A received message: Generator A says hello at 19.
at time 26: Consumer A received message: Generator A says hello at 26.
at time 36: Consumer A received message: Generator A says hello at 36.
at time 46: Consumer A received message: Generator A says hello at 46.
at time 52: Consumer A received message: Generator A says hello at 52.
at time 58: Consumer A received message: Generator A says hello at 58.
LATE Getting Message: at time 66: Consumer A received message: Generator A says hello at 65
at time 75: Consumer A received message: Generator A says hello at 75.
at time 85: Consumer A received message: Generator A says hello at 85.
at time 95: Consumer A received message: Generator A says hello at 95.

One-to-many pipe communication

at time 10: Consumer A received message: G